In [1]:
#!pip install --upgrade transformers datasets scikit-learn


In [2]:
#import sys
#print(sys.executable)

In [3]:
#!{sys.executable} -m pip install --upgrade transformers

In [4]:
from transformers import TrainingArguments
print(TrainingArguments.__doc__)  # Should show evaluation_strategy and save_strategy



    TrainingArguments is the subset of the arguments we use in our example scripts **which relate to the training loop
    itself**.

    Using [`HfArgumentParser`] we can turn this class into
    [argparse](https://docs.python.org/3/library/argparse#module-argparse) arguments that can be specified on the
    command line.

    Parameters:
        output_dir (`str`, *optional*, defaults to `"trainer_output"`):
            The output directory where the model predictions and checkpoints will be written.
        overwrite_output_dir (`bool`, *optional*, defaults to `False`):
            If `True`, overwrite the content of the output directory. Use this to continue training if `output_dir`
            points to a checkpoint directory.
        do_train (`bool`, *optional*, defaults to `False`):
            Whether to run training or not. This argument is not directly used by [`Trainer`], it's intended to be used
            by your training/evaluation scripts instead. See the [example
   

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [6]:
# Load dataset
df = pd.read_csv(r"F:\pbl research\new research paper\Sarcasm_combined.csv")

# Inspect columns
print(df.columns)

# Example: if columns are ['text', 'label'], adjust below accordingly
text_column = 'text'    # replace if different
label_column = 'label'  # replace if different


Index(['tweet_id', 'text', 'label'], dtype='object')


In [7]:
def clean_text(text):
    # Remove unwanted characters, keep letters and spaces
    text = re.sub(r"[^a-zA-Z\s]", "", str(text))
    return text.lower()

df['clean_text'] = df[text_column].apply(clean_text)


In [8]:
# Encode labels as integers
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df[label_column])

# Split train/test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['clean_text'], df['label_encoded'], test_size=0.2, random_state=42, stratify=df['label_encoded']
)


In [9]:
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': list(train_labels)
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': list(test_labels)
})


In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(le.classes_))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="./bert_sarcasm",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)



In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, zero_division=0),
        'recall': recall_score(labels, preds, zero_division=0),
        'f1': f1_score(labels, preds, average='binary')
    }


In [13]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\Charvi\AppData\Local\Temp\ipykernel_41816\1271713805.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# Step 11: Train the model
trainer.train()

C:\Users\Charvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.104550,0.955238,0.695652,0.950495,0.803347
2,0.128600,0.085544,0.959048,0.707143,0.980198,0.821577
3,0.128600,0.078887,0.968571,0.793103,0.910891,0.847926


C:\Users\Charvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Charvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=789, training_loss=0.11480729570860192, metrics={'train_runtime': 6491.4108, 'train_samples_per_second': 1.941, 'train_steps_per_second': 0.122, 'total_flos': 401449914936000.0, 'train_loss': 0.11480729570860192, 'epoch': 3.0})

In [15]:
# Step 12: Evaluate
trainer.evaluate()

C:\Users\Charvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.07888709753751755,
 'eval_accuracy': 0.9685714285714285,
 'eval_precision': 0.7931034482758621,
 'eval_recall': 0.9108910891089109,
 'eval_f1': 0.847926267281106,
 'eval_runtime': 91.0784,
 'eval_samples_per_second': 11.529,
 'eval_steps_per_second': 0.362,
 'epoch': 3.0}